In [1]:
import spacy
import os
import sys

import itertools

import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA

import seaborn as sns
sns.set()
import matplotlib.pyplot as plt

from utils.datasets import load_dataset_from_path, split_train_validation

In [169]:
import json
with open('./datasets/spotify_slice_prova.json') as json_data:
    file = json.load(json_data)
    rows = list()
    for playlist in file['playlists']:
        for track in playlist['tracks']:
            line = list()
            line = [playlist['pid'], playlist['name'], track['track_uri'], track['artist_name'], track['track_name']]
            #print(line)
            rows.append(line)

#print(rows[0])
columns = ['PlaylistPid','PlaylistName','TrackUri', 'ArtistName', 'TrackName']
#print(columns)
df = pd.DataFrame(data = rows, columns = columns)

output_path = 'datasets/Spotify1stSlice.csv'
df.to_csv(output_path, index=False)
df.head()

,PlaylistPid,PlaylistName,TrackUri,ArtistName,TrackName
0,0,Throwbacks,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,Missy Elliott,Lose Control (feat. Ciara & Fat Man Scoop)
1,0,Throwbacks,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,Britney Spears,Toxic
2,0,Throwbacks,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,Beyoncé,Crazy In Love
3,0,Throwbacks,spotify:track:1AWQoqb9bSvzTjaLralEkT,Justin Timberlake,Rock Your Body
4,0,Throwbacks,spotify:track:1lzr43nnXAijIGYnCT8M8H,Shaggy,It Wasn't Me


In [41]:
from utils.dataset_parsing import *
from utils.progress import progress
import utils.sentiment_analysis as sa
import lyricwikia
import io

lyrics_dir = './spotify_lyrics'
rows = list() 
total = len(df)
count = 0

if not os.path.lexists(lyrics_dir):
    os.mkdir(lyrics_dir)

for idx, row in df.iterrows():
    fname = '.'.join([lyrics_dir+'/'+row['TrackUri'],'txt'])
    print(fname)
    if not os.path.lexists(fname):
        try:
            lyrics = lyricwikia.get_lyrics(row['ArtistName'], row['TrackName'])
            with open('.'.join([lyrics_dir+'/'+row['TrackUri'],'txt']), 'w') as f:
                f.write(lyrics)
        except lyricwikia.LyricsNotFound:
            lyrics = None
            continue
        
    if os.path.lexists(fname):
        with io.open(fname, 'r', encoding='utf-8', errors='replace') as f:
            content = f.read()
            lyric_doc = nlp(content)
            lyric = preprocess(content)
            features = feature_extraction(lyric, row['TrackName'])

            freq = features['frequencies'] 
            sentiment = sa.analyse(content)
            elem = (
                row['PlaylistPid'],row['PlaylistName'], row['TrackUri'], row['ArtistName'], row['TrackName'],
                lyric_doc.vector,
                features['line_count'], features['word_count'],#get_line_count(lyric), get_word_count(lyric),
                #get_slang_counts(lyric),
                features['echoisms'], features['selfish'],#get_echoisms(lyric), get_selfish_degree(lyric),
                count_duplicate_lines(lyric), features['is_title_in_lyrics'],# (row['Song'], lyric),
                features['rhymes'],#get_rhymes(lyric),
                features['verb_tenses']['present'], features['verb_tenses']['past'], features['verb_tenses']['future'], #verb_freq['present'], verb_freq['past'], verb_freq['future'],
                freq['ADJ'], freq['ADP'], freq['ADV'], freq['AUX'], freq['CONJ'], 
                freq['CCONJ'], freq['DET'], freq['INTJ'], freq['NOUN'], freq['NUM'],
                freq['PART'], freq['PRON'], freq['PROPN'], freq['PUNCT'], freq['SCONJ'],
                freq['SYM'], freq['VERB'], freq['X'], freq['SPACE'],
                sentiment[0], sentiment[1]
            )
    rows.append(elem)
    count += 1
    progress(count, total, '{}/{}'.format(count, total))

columns = ['PlaylistPid','PlaylistName','TrackUri', 'ArtistName', 'TrackName','LyricVector','lineCount','wordCount',
          'Echoisms','Selfish','DuplicateLines','IsTitleInLyrics','Rhymes','VerbPresent','VerbPast','VerbFuture',
          'ADJ','ADP','ADV','AUX','CONJ','CCONJ','DET','INTJ','NOUN','NUM','PART','PRON','PROPN','PUNCT','SCONJ','SYM',
          'VERB','X','SPACE','Sentiment','Subjectivity']
new_df = pd.DataFrame(data=rows,columns=columns)

output_path = 'datasets/Spotify1stSlice_featurized.csv'
new_df.to_csv(output_path)
print()
print('Done! Dataset written:', output_path)

./spotify_lyrics/spotify:track:0UaMYEvWZi0ZqiDOoHU3YI.txt
./spotify_lyrics/spotify:track:6I9VzXrHxO9rA9A5euc8Ak.txt----] 0.0% ...1/67503
./spotify_lyrics/spotify:track:0WqIKmW4BTrj3eJFmnCKMv.txt----] 0.0% ...2/67503
./spotify_lyrics/spotify:track:1AWQoqb9bSvzTjaLralEkT.txt----] 0.0% ...3/67503
./spotify_lyrics/spotify:track:1lzr43nnXAijIGYnCT8M8H.txt----] 0.0% ...4/67503
./spotify_lyrics/spotify:track:0XUfyU2QviPAs6bxSpXYG4.txt----] 0.0% ...5/67503
./spotify_lyrics/spotify:track:68vgtRHr7iZHpzGpon6Jlo.txt----] 0.0% ...6/67503
./spotify_lyrics/spotify:track:3BxWKCI06eQ5Od8TY2JBeA.txt----] 0.0% ...7/67503
./spotify_lyrics/spotify:track:7H6ev70Weq6DdpZyyTmUXk.txt----] 0.0% ...8/67503
./spotify_lyrics/spotify:track:2PpruBYCo4H7WOBJ7Q2EwM.txt----] 0.0% ...9/67503
./spotify_lyrics/spotify:track:2gam98EZKrF9XuOkU13ApN.txt----] 0.0% ...10/67503
./spotify_lyrics/spotify:track:4Y45aqo9QMa57rDsAJv40A.txt----] 0.0% ...11/67503
./spotify_lyrics/spotify:track:1HwpWwa6bnqqRhK8agG4RS.txt----] 0.0% ...

./spotify_lyrics/spotify:track:4mlqHOqOXja92BT4M51lkj.txt----] 0.3% ...203/67503
./spotify_lyrics/spotify:track:52OKylOQQSETWtXncZN3L7.txt----] 0.3% ...204/67503
./spotify_lyrics/spotify:track:3hyi4CYvZ3K6EqAI2ja7pl.txt----] 0.3% ...205/67503
./spotify_lyrics/spotify:track:3Ex4oMC0UeAQNAG78AuVfi.txt----] 0.3% ...206/67503
./spotify_lyrics/spotify:track:590sE8TWkgAscOaLyjvYRl.txt----] 0.3% ...207/67503
./spotify_lyrics/spotify:track:6tZUEAbX1M8twwJG26nLPf.txt----] 0.3% ...208/67503
./spotify_lyrics/spotify:track:17q3q8wMWgYB8HoqZNNMRT.txt----] 0.3% ...209/67503
./spotify_lyrics/spotify:track:6R7gv0TqoBs4CFDhm5uPzr.txt----] 0.3% ...210/67503
./spotify_lyrics/spotify:track:6AAWJJ0zamfEVReLLDj4BC.txt----] 0.3% ...211/67503
./spotify_lyrics/spotify:track:62Rq7yIwT6p2FFxiszk3dc.txt----] 0.3% ...212/67503
./spotify_lyrics/spotify:track:1gMxEMadnj1DIiny9wOAiT.txt----] 0.3% ...213/67503
./spotify_lyrics/spotify:track:1KVU56xCG8SqaXRjf9n4Da.txt----] 0.3% ...214/67503
./spotify_lyrics/spotify:tra

./spotify_lyrics/spotify:track:4SlGszCL8Kxamm9ngmA1tJ.txt----] 0.6% ...405/67503
./spotify_lyrics/spotify:track:6Wt5QBzHp9vdSLAFlvdnDQ.txt----] 0.6% ...406/67503
./spotify_lyrics/spotify:track:3cdcT7I1xnQqQ84zMCtrKU.txt----] 0.6% ...407/67503
./spotify_lyrics/spotify:track:4B1Gq9foMAhAgkAaLTSaTl.txt----] 0.6% ...408/67503
./spotify_lyrics/spotify:track:2WQn7Yvs728KZmmY6tgWqH.txt----] 0.6% ...409/67503
./spotify_lyrics/spotify:track:33wIYOxUVvm1AahfX3e5aC.txt----] 0.6% ...410/67503
./spotify_lyrics/spotify:track:57awyWlR4Itwb2DFIFmd9i.txt----] 0.6% ...411/67503
./spotify_lyrics/spotify:track:1EqLuIHIWIoFt1xAFXGZSC.txt----] 0.6% ...412/67503
./spotify_lyrics/spotify:track:3kxfsdsCpFgN412fpnW85Y.txt----] 0.6% ...413/67503
./spotify_lyrics/spotify:track:3zgs1f6Tj0mn29qTXJuhSY.txt----] 0.6% ...414/67503
./spotify_lyrics/spotify:track:0VaIsQoIQY10uxBrOyZ5cQ.txt----] 0.6% ...415/67503
./spotify_lyrics/spotify:track:30msOgLWuLnrO0MWIG4ifE.txt----] 0.6% ...416/67503
./spotify_lyrics/spotify:tra

./spotify_lyrics/spotify:track:2GHCGJ8LSmSy3GDnGwPssh.txt----] 0.9% ...607/67503
./spotify_lyrics/spotify:track:06ZumdD89viAIlyPlaW3hX.txt----] 0.9% ...608/67503
./spotify_lyrics/spotify:track:2s8SzxTOJM6qUrxKmou4BK.txt----] 0.9% ...609/67503
./spotify_lyrics/spotify:track:0GsQ4dS8a6vV3P3A7lc4nc.txt----] 0.9% ...610/67503
./spotify_lyrics/spotify:track:5vQq4voQxd3QTkkdkONX1Q.txt----] 0.9% ...611/67503
./spotify_lyrics/spotify:track:61KHfpOOqUgSWpzfpufkzs.txt----] 0.9% ...612/67503
./spotify_lyrics/spotify:track:3kKVqFF4pv4EXeQe428zl2.txt----] 0.9% ...613/67503
./spotify_lyrics/spotify:track:4bWnKmUeVwQIM0n2QwZ5Sc.txt----] 0.9% ...614/67503
./spotify_lyrics/spotify:track:0lNv8HyonQ5c6SYwkcmwD8.txt----] 0.9% ...615/67503
./spotify_lyrics/spotify:track:55SAyrUcLo8fEFjvcN2twt.txt----] 0.9% ...616/67503
./spotify_lyrics/spotify:track:7vBhC6oWcqP8ug24Ctz7Su.txt----] 0.9% ...617/67503
./spotify_lyrics/spotify:track:5Nn2Dj7OQsGL6pgQ9iIzPp.txt----] 0.9% ...618/67503
./spotify_lyrics/spotify:tra

./spotify_lyrics/spotify:track:30HCB1FoE77IfGRyNv4eFq.txt----] 1.2% ...809/67503
./spotify_lyrics/spotify:track:2YodwKJnbPyNKe8XXSE9V7.txt----] 1.2% ...810/67503
./spotify_lyrics/spotify:track:4P5KoWXOxwuobLmHXLMobV.txt----] 1.2% ...811/67503
./spotify_lyrics/spotify:track:020gYw7juUaNs2tLYMlfms.txt----] 1.2% ...812/67503
./spotify_lyrics/spotify:track:36bOT34EdRNrNhCWMwNJKu.txt----] 1.2% ...813/67503
./spotify_lyrics/spotify:track:5PsUB0ISfQxRLhF5DmI5Ks.txt----] 1.2% ...814/67503
./spotify_lyrics/spotify:track:5qqabIl2vWzo9ApSC317sa.txt----] 1.2% ...815/67503
./spotify_lyrics/spotify:track:1wo3UYTeizJHkwYIuLuBPF.txt----] 1.2% ...816/67503
./spotify_lyrics/spotify:track:51T1wGHtx6hXW89UCzw1Rc.txt----] 1.2% ...817/67503
./spotify_lyrics/spotify:track:62nQ8UZVqR2RMvkJHkcO2o.txt----] 1.2% ...818/67503
./spotify_lyrics/spotify:track:2B98ljvzqpCVgt5reTHq28.txt----] 1.2% ...819/67503
./spotify_lyrics/spotify:track:2wVXSc0IrQVodHESUxVUJA.txt----] 1.2% ...820/67503
./spotify_lyrics/spotify:tra

./spotify_lyrics/spotify:track:76EeScTnI2sCjDY0SfEoSb.txt----] 1.5% ...1011/67503
./spotify_lyrics/spotify:track:3e0yTP5trHBBVvV32jwXqF.txt----] 1.5% ...1012/67503
./spotify_lyrics/spotify:track:1aGvLFHJ2shKqO9uycaUcW.txt----] 1.5% ...1013/67503
./spotify_lyrics/spotify:track:6zQyu8L8yUuJkl6LbQ6iKU.txt----] 1.5% ...1014/67503
./spotify_lyrics/spotify:track:5mhUPDWQH3s544WCsa0r9w.txt----] 1.5% ...1015/67503
./spotify_lyrics/spotify:track:7oK9VyNzrYvRFo7nQEYkWN.txt----] 1.5% ...1016/67503
./spotify_lyrics/spotify:track:33lXq3csEWwYNn6x0sM3Et.txt----] 1.5% ...1017/67503
./spotify_lyrics/spotify:track:756CJtQRFSxEx9jV4P9hpA.txt----] 1.5% ...1018/67503
./spotify_lyrics/spotify:track:5xkNd7kCRmjT2DxSAYVVyC.txt----] 1.5% ...1019/67503
./spotify_lyrics/spotify:track:4Z1fbYp0HuxLBje4MOZcSD.txt----] 1.5% ...1020/67503
./spotify_lyrics/spotify:track:0KS9V6ihDVgbotoOov3QZH.txt----] 1.5% ...1021/67503
./spotify_lyrics/spotify:track:1SSmCRKVovLanPi3mx3pVe.txt----] 1.5% ...1022/67503
./spotify_lyrics

./spotify_lyrics/spotify:track:0QnONzv3TvHAWk294h6DaQ.txt----] 1.8% ...1209/67503
./spotify_lyrics/spotify:track:3ikZyuwdbrWeGPi1rTp1Kp.txt----] 1.8% ...1210/67503
./spotify_lyrics/spotify:track:1zByJfArrVwsdJ4C2WdegY.txt----] 1.8% ...1211/67503
./spotify_lyrics/spotify:track:28cnXtME493VX9NOw9cIUh.txt----] 1.8% ...1212/67503
./spotify_lyrics/spotify:track:3kZC0ZmFWrEHdUCmUqlvgZ.txt----] 1.8% ...1213/67503
./spotify_lyrics/spotify:track:0B7wvvmu9EISAwZnOpjhNI.txt----] 1.8% ...1214/67503
./spotify_lyrics/spotify:track:2obblQ6tcePeOEVJV6nEGD.txt----] 1.8% ...1215/67503
./spotify_lyrics/spotify:track:5YjKdeES9QRJ8NmF4Xc8pV.txt----] 1.8% ...1216/67503
./spotify_lyrics/spotify:track:3VyJPsb2ucaaVptLSDpJNH.txt----] 1.8% ...1217/67503
./spotify_lyrics/spotify:track:06u5LrUpbosQlQ1QJFhPpG.txt----] 1.8% ...1218/67503
./spotify_lyrics/spotify:track:6PmnGYDsruYLBNY4Rpx4t9.txt----] 1.8% ...1219/67503
./spotify_lyrics/spotify:track:60kqoWSXSbiDJaDxgi8GG1.txt----] 1.8% ...1220/67503
./spotify_lyrics

./spotify_lyrics/spotify:track:1hWYT0w2R0J19rlVkiez7X.txt----] 2.1% ...1407/67503
./spotify_lyrics/spotify:track:2gZUPNdnz5Y45eiGxpHGSc.txt----] 2.1% ...1408/67503
./spotify_lyrics/spotify:track:7lxZbmtC6SkUc2yXBgDOdO.txt----] 2.1% ...1409/67503
./spotify_lyrics/spotify:track:55OdqrG8WLmsYyY1jijD9b.txt----] 2.1% ...1410/67503
./spotify_lyrics/spotify:track:6M0IsaUX4GNyto4niSegfI.txt----] 2.1% ...1411/67503
./spotify_lyrics/spotify:track:4KW1lqgSr8TKrvBII0Brf8.txt----] 2.1% ...1412/67503
./spotify_lyrics/spotify:track:1fkMGqxmsofVygRqah5OuK.txt----] 2.1% ...1413/67503
./spotify_lyrics/spotify:track:4cg1yakyRSIOjxKM2I7J1q.txt----] 2.1% ...1414/67503
./spotify_lyrics/spotify:track:1FsH6OZCKLWbBgqj8JcCvO.txt----] 2.1% ...1415/67503
./spotify_lyrics/spotify:track:1uUGn3Sk4qErHdDqL1OSVA.txt----] 2.1% ...1416/67503
./spotify_lyrics/spotify:track:2nBI3iWLhupR7LyAJ5GGkE.txt----] 2.1% ...1417/67503
./spotify_lyrics/spotify:track:23SZWX2IaDnxmhFsSLvkG2.txt----] 2.1% ...1418/67503
./spotify_lyrics

HTTPError: HTTP Error 503: SERVICE UNAVAILABLE

In [47]:
columns = ['PlaylistPid','PlaylistName','TrackUri', 'ArtistName', 'TrackName','LyricVector','lineCount','wordCount',
          'Echoisms','Selfish','DuplicateLines','IsTitleInLyrics','Rhymes','VerbPresent','VerbPast','VerbFuture',
          'ADJ','ADP','ADV','AUX','CONJ','CCONJ','DET','INTJ','NOUN','NUM','PART','PRON','PROPN','PUNCT','SCONJ','SYM',
          'VERB','X','SPACE','PosSentiment','NeutralSentiment','NegSentiment']
new_df = pd.DataFrame(data=rows,columns=columns)
print(len(new_df))
new_df.head()
output_path = 'datasets/spotify_lyrics_1st.csv'
new_df.to_csv(output_path)
print()
print('Done! Dataset written:', output_path)

1571

Done! Dataset written: datasets/spotify_lyrics_1st.csv


In [2]:
dataset = pd.read_csv('./datasets/moodylyrics_featurized.csv')
#print(dataset.columns)

useless_columns = [ 'ID','ARTIST', 'SONG_TITLE', 'EMOTION', 'X_FREQUENCIES', 'SPACE_FREQUENCIES']
selected_columns = [
   'LYRICS_VECTOR',
   'WORD_COUNT', 'ECHOISMS', 'SELFISH_DEGREE', 
   'DUPLICATE_LINES'
    , 'IS_TITLE_IN_LYRICS', 'VERB_PRESENT', 
   'VERB_PAST', 'VERB_FUTURE', 'ADJ_FREQUENCIES',
   'PUNCT_FREQUENCIES','SENTIMENT','SUBJECTIVITY',
   'EMOTION'
]
dataset = dataset[selected_columns]
tmp_df = dataset.drop(['EMOTION'], axis=1)
X_vect = list()
for (i, row) in tmp_df.iterrows():
    sub_list = list()
    for field in row:
        if type(field) == str:
            field = field[1:-1].split()
            sub_list += [float(x.replace('\n','')) for x in field]
        else:
            sub_list.append(field)
    X_vect.append(np.array(sub_list))
X_vect = np.array(X_vect)

y = dataset['EMOTION'].as_matrix().T
print(X_vect.shape)
print(y.shape)


(2452, 312)
(2452,)


In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_vect, y, test_size = 0.2, random_state = 0)

In [4]:
#1 Importing the Keras libraries and packages
import keras
# Sequential module is required to initialize our ANN
from keras.models import Sequential
# Dense module is required to create the layers
from keras.layers import Dense, Dropout
    
def build_ann(optimizer='adam', input_size=X_vect.shape[1]):
    classifier = Sequential()
    #2 Adding first hidden layer
    classifier.add(Dense(units = 60, kernel_initializer = 'random_normal', activation = 'sigmoid', input_dim = input_size))
    classifier.add(Dropout(0.5))
    
    #2 Adding second hidden layer
    classifier.add(Dense(units = 60, kernel_initializer = 'random_normal', activation = 'sigmoid', input_dim = input_size))
    classifier.add(Dropout(0.5))

    # Adding output layer
    classifier.add(Dense(units = 4, kernel_initializer = 'random_normal', activation = 'softmax'))

    #3 Compiling the ANN
    classifier.compile(optimizer=optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy'])

    return classifier

Using TensorFlow backend.


In [5]:
from keras.utils import np_utils
encoder = LabelEncoder()
y_nn = np_utils.to_categorical(encoder.fit_transform(y_train))

#3 Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
# we need to scale because we don't want one feature to predomine the others
# Standardize features by removing the mean and scaling to unit variance
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

classifier = build_ann()
classifier.fit(X_train, y_nn, batch_size = 64, epochs = 100, verbose=0)

#1 Predicting the Test set results
y_pred = classifier.predict(X_test,verbose=0)
y_pred_index = np.argmax(y_pred,axis=1)

encoder = LabelEncoder()
y_nn_pred = np_utils.to_categorical(encoder.fit_transform(y_test))

cm = confusion_matrix(y_pred_index, y_nn_pred.argmax(axis=1))
accuracy = (sum([cm[i,i] for i in range(len(cm))])) / len(y_nn_pred)
print('Accuracy: %0.2f' % (accuracy*100))

Accuracy: 90.02


In [6]:
new_df = pd.read_csv('./datasets/spotify_lyrics_1st.csv')
selected_columns = ['PlaylistPid', 'PlaylistName', 'TrackUri', 'ArtistName', 'TrackName',
       'LyricVector', 'wordCount', 'Echoisms', 'Selfish',
       'DuplicateLines', 'IsTitleInLyrics', 'VerbPresent',
       'VerbPast', 'VerbFuture', 'ADJ', 'PUNCT'
]

new_df = new_df[selected_columns]
tmp_df = new_df.drop(['PlaylistPid', 'PlaylistName', 'TrackUri', 'ArtistName', 'TrackName'], axis=1)

X_s = list()
for (i, row) in tmp_df.iterrows():
    sub_list = list()
    for field in row:
        if type(field) == str:
            field = field[1:-1].split()
            sub_list += [float(x.replace('\n','')) for x in field]
        else:
            sub_list.append(field)
    X_s.append(np.array(sub_list))
X_s = np.array(X_s)

print(X_s.shape)

FileNotFoundError: File b'./datasets/spotify_lyrics_1st.csv' does not exist

In [112]:
new_df.head()

,PlaylistPid,PlaylistName,TrackUri,ArtistName,TrackName,LyricVector,wordCount,Echoisms,Selfish,DuplicateLines,IsTitleInLyrics,VerbPresent,VerbPast,VerbFuture,ADJ,PUNCT
0,0,Throwbacks,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,Missy Elliott,Lose Control (feat. Ciara & Fat Man Scoop),[-2.87126824e-02 1.77806959e-01 -1.26087472e-...,503,0.0,0.102941,0.093439,False,0.866667,0.100000,0.033333,0.083499,0.349901
1,0,Throwbacks,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,Britney Spears,Toxic,[-2.08647419e-02 1.90544352e-01 -2.25056827e-...,276,0.0,0.358209,0.072464,False,0.981132,0.018868,0.000000,0.105072,0.134058
2,0,Throwbacks,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,Beyoncé,Crazy In Love,[-3.40463594e-02 1.83713183e-01 -2.36143962e-...,693,0.0,0.213115,0.053391,False,0.623377,0.324675,0.051948,0.113997,0.079365
3,0,Throwbacks,spotify:track:1AWQoqb9bSvzTjaLralEkT,Justin Timberlake,Rock Your Body,[-1.64101850e-02 1.44258231e-01 -2.49298677e-...,498,0.0,0.244444,0.060241,False,0.884058,0.086957,0.028986,0.048193,0.056225
4,0,Throwbacks,spotify:track:1lzr43nnXAijIGYnCT8M8H,Shaggy,It Wasn't Me,[-2.05015372e-02 1.49805278e-01 -1.77448347e-...,584,0.0,0.190909,0.075342,False,0.490909,0.509091,0.000000,0.075342,0.085616


In [151]:
X_s_scaled = sc.transform(X_s)
emotions_label = ['happy', 'sad', 'angry', 'relaxed']
#1 Predicting the Test set results
y_pred_s = classifier.predict(X_s_scaled,verbose=0)
y_prova = [list(zip(y_pred_s[i], emotions_label)) for i in range(len(y_pred_s))]

y_prova_ord = [sorted(y_prova[i], key=lambda x:  x[0], reverse=True) for i in range(len(y_prova))]

#print(y_prova[:2])

y_pred_1 = np.argmax(y_pred_s,axis=1)

for i in range(52):
    print(new_df['TrackName'][i], '-->' ,[(y[1],round(y[0],2)) for y in y_prova_ord[i]])


Lose Control (feat. Ciara & Fat Man Scoop) --> [('relaxed', 0.77), ('happy', 0.21), ('angry', 0.01), ('sad', 0.0)]
Toxic --> [('happy', 0.89), ('relaxed', 0.11), ('angry', 0.0), ('sad', 0.0)]
Crazy In Love --> [('happy', 0.64), ('sad', 0.3), ('relaxed', 0.06), ('angry', 0.0)]
Rock Your Body --> [('angry', 0.98), ('relaxed', 0.01), ('sad', 0.0), ('happy', 0.0)]
It Wasn't Me --> [('relaxed', 0.81), ('angry', 0.15), ('happy', 0.04), ('sad', 0.0)]
Yeah! --> [('happy', 0.99), ('relaxed', 0.01), ('angry', 0.0), ('sad', 0.0)]
My Boo --> [('sad', 0.93), ('angry', 0.06), ('happy', 0.01), ('relaxed', 0.0)]
Buttons --> [('relaxed', 0.91), ('happy', 0.07), ('angry', 0.02), ('sad', 0.0)]
Buttons --> [('relaxed', 0.91), ('happy', 0.07), ('angry', 0.02), ('sad', 0.0)]
Buttons --> [('relaxed', 0.91), ('happy', 0.07), ('angry', 0.02), ('sad', 0.0)]
Promiscuous --> [('angry', 0.83), ('sad', 0.09), ('happy', 0.05), ('relaxed', 0.03)]
Promiscuous --> [('angry', 0.83), ('sad', 0.09), ('happy', 0.05), ('rel

In [166]:
playlist1 = np.zeros((52, 4))
playlist1 = [[y_prova[i][j][0] for j in range(4)] for i in range(52) ]
#print(playlist1)

In [167]:
result = np.sum(playlist1, axis=0)
result = result/52
print(result)


[0.21998337 0.17509787 0.2861372  0.31878152]
